# Предсказание только по начальным данным

In [1]:
import pandas as pd
import numpy as np
features = pd.read_csv(r'S:\Coursera\Введение в машинное обучение\Итоговое задание\features.csv', index_col='match_id')

In [2]:
pd.set_option('display.max_columns', None)
features

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_hero,r2_level,r2_xp,r2_gold,r2_lh,r2_kills,r2_deaths,r2_items,r3_hero,r3_level,r3_xp,r3_gold,r3_lh,r3_kills,r3_deaths,r3_items,r4_hero,r4_level,r4_xp,r4_gold,r4_lh,r4_kills,r4_deaths,r4_items,r5_hero,r5_level,r5_xp,r5_gold,r5_lh,r5_kills,r5_deaths,r5_items,d1_hero,d1_level,d1_xp,d1_gold,d1_lh,d1_kills,d1_deaths,d1_items,d2_hero,d2_level,d2_xp,d2_gold,d2_lh,d2_kills,d2_deaths,d2_items,d3_hero,d3_level,d3_xp,d3_gold,d3_lh,d3_kills,d3_deaths,d3_items,d4_hero,d4_level,d4_xp,d4_gold,d4_lh,d4_kills,d4_deaths,d4_items,d5_hero,d5_level,d5_xp,d5_gold,d5_lh,d5_kills,d5_deaths,d5_items,first_blood_time,first_blood_team,first_blood_player1,first_blood_player2,radiant_bottle_time,radiant_courier_time,radiant_flying_courier_time,radiant_tpscroll_count,radiant_boots_count,radiant_ward_observer_count,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,67,3,842,991,10,0,0,4,29,5,1909,1143,10,0,0,8,20,3,757,741,6,0,0,7,105,3,732,658,4,0,1,11,4,3,1058,996,12,0,0,6,42,4,1085,986,12,0,0,4,21,5,2052,1536,23,0,0,6,37,3,742,500,2,0,0,8,84,3,958,1003,3,1,0,9,7.0,1.0,9.0,NaN,134.0,-80.0,244.0,2,2,2,0,35.0,103.0,-84.0,221.0,3,4,2,2,-52.0,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,49,4,1596,993,10,0,1,7,67,4,1506,1502,18,1,0,7,37,3,669,631,7,0,0,7,26,2,415,539,1,0,0,5,39,5,1960,1384,16,0,0,8,88,3,640,566,1,0,1,5,79,3,720,1350,2,2,0,12,7,2,440,583,0,0,0,7,12,4,1470,1622,24,0,0,9,54.0,1.0,7.0,NaN,173.0,-80.0,NaN,2,0,2,0,-20.0,149.0,-84.0,195.0,5,4,3,1,-5.0,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,98,3,1314,775,6,0,0,6,20,3,1297,909,0,1,0,6,27,5,2360,2096,26,1,1,6,4,3,1395,1627,27,0,0,9,22,5,2305,2028,19,1,1,10,66,3,1024,959,19,0,1,10,86,3,755,620,3,0,0,8,29,4,1319,667,4,0,0,7,80,3,1350,1512,25,0,0,7,224.0,0.0,3.0,NaN,63.0,-82.0,NaN,2,5,2,1,-39.0,45.0,-77.0,221.0,3,4,3,1,13.0,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,30,2,539,539,1,0,0,6,75,5,2037,1139,15,0,0,6,37,2,591,499,0,0,0,6,41,3,712,1075,12,0,0,6,96,5,1878,1174,17,0,0,6,48,3,732,1468,22,0,0,10,15,4,1681,1051,11,0,0,7,102,2,674,537,1,0,0,7,20,2,510,499,0,0,0,7,NaN,NaN,NaN,NaN,208.0,-75.0,NaN,0,3,2,0,-30.0,124.0,-80.0,184.0,0,4,2,0,27.0,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,27,2,629,552,0,0,1,7,30,3,884,927,0,1,0,8,72,3,925,1439,16,1,0,11,93,4,1482,880,7,0,0,8,26,3,704,586,1,0,2,9,69,3,1169,1665,20,1,0,7,22,3,1055,638,1,0,0,9,25,5,1815,1275,18,0,0,8,8,4,1119,904,6,0,1,7,-21.0,1.0,6.0,NaN,166.0,-81.0,181.0,1,4,2,0,46.0,182.0,-80.0,225.0,6,3,3,0,-16.0,2449,0,4,1974,3,63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114402,1450265551,1,47,4,1706,1198,17,0,1,8,7,2,616,535,0,0,2,10,1,3,879,1798,22,1,1,9,21,4,1835,1350,11,1,1,9,71,3,604,567,1,0,1,5,26,3,887,1023,0,1,0,7,19,5,2192,1674,13,2,0,7,93,5,1637,1339,18,0,1,9,3,3,972,847,1,1,1,11,28,3,1583,1634,10,2,0,6,114.0,0.0,2.0,8.0,245.0,-86.0,211.0,5,2,1,0,-29.0,180.0,-76.0,180.0,3,4,3,0,-24.0,2032,0,1792,1975,48,63
114403,1450277704,0,43,4,1793,1416,17,0,1,5,26,3,764,800,0,1,0,9,4,3,1130,1585,20,1,1,14,29,4,1546,1166,8,1,0,6,110,3,892,541,1,0,1,4,72,3,966,1303,17,0,1,6,75,5,1745,1002,4,1,0,6,5,2,308

In [3]:
#колонки с категориальными признаками
cat_cols = ['lobby_type',
            'r1_hero',
           'r2_hero',
           'r3_hero',
           'r4_hero',
           'r5_hero',
           'd1_hero',
           'd2_hero',
           'd3_hero',
           'd4_hero',
           'd5_hero'
           ]

In [4]:
#колонки с категориальными и числовыми признаками
feature_cols = cat_cols

#целевая переменная
target_col = ['radiant_win']

In [5]:
#функция выделяет из массива данных X и y

def create_y_X(data):
    y = data[target_col]
    X = data[feature_cols]

    return y, X

In [6]:
y, X = create_y_X(features)

In [7]:
X.corr().style.set_precision(2)

#В матрице корреляций можно увидеть достаточно большую корреляцию между признаками уровня, 
#опыта, ценности героя и числом убитых юнитов. В дальнейшем это можно использовать для
#создания новых признаков.

,lobby_type,r1_hero,r2_hero,r3_hero,r4_hero,r5_hero,d1_hero,d2_hero,d3_hero,d4_hero,d5_hero
lobby_type,1.00,-0.01,-0.00,-0.01,-0.01,-0.01,-0.01,-0.01,-0.02,-0.01,-0.01
r1_hero,-0.01,1.00,-0.02,-0.02,-0.02,-0.01,-0.02,-0.02,-0.03,-0.03,-0.02
r2_hero,-0.00,-0.02,1.00,-0.02,-0.01,-0.02,-0.03,-0.03,-0.02,-0.03,-0.02
r3_hero,-0.01,-0.02,-0.02,1.00,-0.02,-0.02,-0.03,-0.02,-0.02,-0.03,-0.03
r4_hero,-0.01,-0.02,-0.01,-0.02,1.00,-0.02,-0.03,-0.02,-0.02,-0.03,-0.02
r5_hero,-0.01,-0.01,-0.02,-0.02,-0.02,1.00,-0.03,-0.03,-0.03,-0.02,-0.02
d1_hero,-0.01,-0.02,-0.03,-0.03,-0.03,-0.03,1.00,-0.02,-0.02,-0.02,-0.02
d2_hero,-0.01,-0.02,-0.03,-0.02,-0.02,-0.03,-0.02,1.00,-0.02,-0.02,-0.02
d3_hero,-0.02,-0.03,-0.02,-0.02,-0.02,-0.03,-0.02,-0.02,1.00,-0.02,-0.02
d4_hero,-0.01,-0.03,-0.03,-0.03,-0.03,-0.02,-0.02,-0.02,-0.02,1.00,-0.02


In [8]:
X

,lobby_type,r1_hero,r2_hero,r3_hero,r4_hero,r5_hero,d1_hero,d2_hero,d3_hero,d4_hero,d5_hero
match_id,,,,,,,,,,,
0,7,11,67,29,20,105,4,42,21,37,84
1,0,42,49,67,37,26,39,88,79,7,12
2,7,33,98,20,27,4,22,66,86,29,80
3,1,29,30,75,37,41,96,48,15,102,20
4,7,13,27,30,72,93,26,69,22,25,8
...,...,...,...,...,...,...,...,...,...,...,...
114402,1,47,7,1,21,71,26,19,93,3,28
114403,0,43,26,4,29,110,72,75,5,20,98
114404,1,98,11,112,81,50,28,39,55,59,31


# Градиентный бустинг 
Описание: игнорируем категориальные фичи

In [9]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import roc_auc_score

In [10]:
y = np.array(y)
y = y.ravel()

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=2)
gbc = GradientBoostingClassifier(random_state=2)

In [12]:
gbc.fit(X_train, y_train)

GradientBoostingClassifier(random_state=2)

In [13]:
Pred_gbc_test = gbc.predict_proba(X_test)
Pred_gbc_train = gbc.predict_proba(X_train)
roc_auc_test = roc_auc_score(y_test, Pred_gbc_test[:,1])
roc_auc_train = roc_auc_score(y_train, Pred_gbc_train[:,1])
print('значение roc_auc на test: ', roc_auc_test)
print('значение roc_auc на train: ', roc_auc_train)
print('среднее значение roc_auc: ', (roc_auc_test + roc_auc_train)/2)

значение roc_auc на test:  0.5580096041487621
значение roc_auc на train:  0.595882160860132
среднее значение roc_auc:  0.5769458825044471


In [17]:
from sklearn.metrics import accuracy_score

In [15]:
y_pred_test = gbc.predict(X_test)
y_pred_train = gbc.predict(X_train)

acc_test = accuracy_score(y_test, y_pred_test)
acc_train = accuracy_score(y_train, y_pred_train)

print('значение acc на test: ', acc_test)
print('значение acc на train: ', acc_train)
print('среднее значение acc: ', (acc_test + acc_train)/2)

значение acc на test:  0.5424251774143782
значение acc на train:  0.567461541852162
среднее значение acc:  0.5549433596332701


In [16]:
X_try = [[1, 47, 85, 56, 7, 52, 106, 29, 15, 101, 87]]
print('radiant win' if gbc.predict(X_try)==1 else 'dire win')
print('вероятность выигрыша radiant', gbc.predict_proba(X_try)[:,1])

radiant win
вероятность выигрыша radiant [0.66421237]


In [17]:
max = 0
for i in range(97230):
    arr = X.iloc[i, :]
    arr = np.expand_dims(arr, axis=0)
    y_pred = gbc.predict_proba(arr)[:, 1]
    if y_pred > max:
        max = y_pred
        arr_max = arr
        
print(max, arr_max)

[0.66421237] [[  1  47  85  56   7  52 106  29  15 101  87]]


In [23]:
heroes = pd.read_csv(r'S:\Coursera\Введение в машинное обучение\Итоговое задание\data\data\dictionaries\heroes.csv', index_col='id')
lobbies  = pd.read_csv(r'S:\Coursera\Введение в машинное обучение\Итоговое задание\data\data\dictionaries\lobbies.csv', index_col='id')

# CatBoost

In [10]:
!pip install catboost
import catboost

In [11]:
#Сейчас некоторые кат. признаки типа float. Напишем функцию, преобразующую их в int

def cat_to_int(data, cat_cols):
    data[cat_cols] = data[cat_cols].astype('int64')
    return data

In [12]:
#Создадим матрицу признаков для кэтбустинга

X_catboost = cat_to_int(X, cat_cols)

C:\Users\Wised\anaconda3\lib\site-packages\pandas\core\frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [13]:
#дефолтный кэтбустинг
cat = catboost.CatBoostClassifier(eval_metric='AUC', silent=True, random_state=2, cat_features=cat_cols)

X_catboost_train, X_catboost_test, y_train, y_test = train_test_split(X_catboost, y, train_size=0.7, random_state=2)


In [14]:
#обучение
cat.fit(X_catboost_train, y_train)

In [15]:
#предсказание

Pred_catboost_test = cat.predict_proba(X_catboost_test)
Pred_catboost_train = cat.predict_proba(X_catboost_train)
roc_auc_test = roc_auc_score(y_test, Pred_catboost_test[:,1])
roc_auc_train = roc_auc_score(y_train, Pred_catboost_train[:,1])
print('значение roc_auc на test: ', roc_auc_test)
print('значение roc_auc на train: ', roc_auc_train)
print('среднее значение roc_auc: ', (roc_auc_test + roc_auc_train)/2)

значение roc_auc на test:  0.583719011730885
значение roc_auc на train:  0.7843276658068478
среднее значение roc_auc:  0.6840233387688663


In [18]:
y_pred_test = cat.predict(X_catboost_test)
y_pred_train = cat.predict(X_catboost_train)

acc_test = accuracy_score(y_test, y_pred_test)
acc_train = accuracy_score(y_train, y_pred_train)

print('значение acc на test: ', acc_test)
print('значение acc на train: ', acc_train)
print('среднее значение acc: ', (acc_test + acc_train)/2)

значение acc на test:  0.5587781548904659
значение acc на train:  0.704838306813006
среднее значение acc:  0.6318082308517359


In [33]:
#Важность признаков

cat.get_feature_importance()

array([ 8.26810572,  9.34655866, 10.09609662,  8.69542929,  9.6001955 ,
        9.00242556,  9.1482491 ,  9.59239559,  9.26364035,  8.16946368,
        8.81743993])

In [107]:
#Лучшая комбинация игроков

max = 0
for i in range(97230):
    arr = X_catboost.iloc[i, :]
    arr = np.expand_dims(arr, axis=0)
    y_pred = cat.predict_proba(arr)[:, 1]
    if y_pred > max:
        max = y_pred
        arr_max = arr
        
print(max, arr_max)

[0.84614805] [[ 1 52 42 79 92 71 96 39 25 86 18]]


In [108]:
print(arr_max)
heroes_number = arr_max[0, 1:]
heroes_number

[[ 1 52 42 79 92 71 96 39 25 86 18]]


array([52, 42, 79, 92, 71, 96, 39, 25, 86, 18], dtype=int64)

In [46]:
for hero_number in heroes_number:
    for i in range(112):
        if hero_number == heroes.index[i]:
            print(heroes.iloc[i, 1])

leshrac
skeleton_king
shadow_demon
visage
spirit_breaker
centaur
queenofpain
lina
rubick
sven


In [19]:
#Онлайн-предсказание

#Вбить персов:
lobby_type = 'Tournament'
r1_hero = 'shadow_demon'
r2_hero = 'templar_assassin'
r3_hero = 'disruptor'
r4_hero = 'sand_king'
r5_hero = 'juggernaut'
d1_hero = 'luna'
d2_hero = 'viper'
d3_hero = 'earth_spirit'
d4_hero = 'spectre'
d5_hero = 'death_prophet'

In [20]:
X_try = [lobby_type,
         r1_hero,
           r2_hero,
           r3_hero,
           r4_hero,
           r5_hero,
           d1_hero,
           d2_hero,
           d3_hero,
           d4_hero,
           d5_hero
           ]

In [28]:
#Номер комнаты:

lobby_type_number = 5

In [29]:
#Получить номера игроков по их названиям:

X_try_numbers = [lobby_type_number]
for hero in X_try:
    for i in range(112):
        if heroes.iloc[i, 1] == hero:
            hero_number = heroes.index[i]
            X_try_numbers.append(hero_number)

In [30]:
X_try_numbers

[5, 79, 46, 87, 16, 8, 48, 47, 107, 67, 43]

In [31]:
#Проверка списка для предсказания:

assert len(X_try_numbers) == 11
for i in X_try_numbers:
    assert i != 0
X_try_numbers

[5, 79, 46, 87, 16, 8, 48, 47, 107, 67, 43]

In [32]:
print('Unique wins' if cat.predict([X_try_numbers])==1 else 'PuckChamp wins')
print('вероятность выигрыша Unique', cat.predict_proba([X_try_numbers])[:,1])

PuckChamp wins
вероятность выигрыша Unique [0.3944561]


ПРЕДСКАЗАНИЯ ДЛЯ ГРАДИЕНТНЫХ БУСТИНГОВ

In [60]:
best_model = cat_grid

In [80]:
X_test_test = pd.read_csv(r'S:\Coursera\Введение в машинное обучение\Итоговое задание\features_test.csv', index_col='match_id')
X_test_test = X_test_test[feature_cols]
X_test_test['first_blood_player1'] = X_test_test['first_blood_player1'].fillna(0)
X_test_test['first_blood_player2'] = X_test_test['first_blood_player2'].fillna(0)
X_test_test['first_blood_team'] = X_test_test['first_blood_team'].fillna(.5)
X_test_test[time_gap_cols] = X_test_test[time_gap_cols].fillna(1000)
X_test_test = cat_to_int(X_test_test, cat_cols)

X_test_test

,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_level,r2_xp,r2_gold,r2_lh,r2_kills,r2_deaths,r2_items,r3_level,r3_xp,r3_gold,r3_lh,r3_kills,r3_deaths,r3_items,r4_level,r4_xp,r4_gold,r4_lh,r4_kills,r4_deaths,r4_items,r5_level,r5_xp,r5_gold,r5_lh,r5_kills,r5_deaths,r5_items,d1_level,d1_xp,d1_gold,d1_lh,d1_kills,d1_deaths,d1_items,d2_level,d2_xp,d2_gold,d2_lh,d2_kills,d2_deaths,d2_items,d3_level,d3_xp,d3_gold,d3_lh,d3_kills,d3_deaths,d3_items,d4_level,d4_xp,d4_gold,d4_lh,d4_kills,d4_deaths,d4_items,d5_level,d5_xp,d5_gold,d5_lh,d5_kills,d5_deaths,d5_items,first_blood_time,first_blood_team,radiant_bottle_time,radiant_courier_time,radiant_flying_courier_time,radiant_tpscroll_count,radiant_boots_count,radiant_ward_observer_count,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,lobby_type,r1_hero,r2_hero,r3_hero,r4_hero,r5_hero,d1_hero,d2_hero,d3_hero,d4_hero,d5_hero,first_blood_player1,first_blood_player2
match_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
6,4,1103,1089,8,0,1,9,3,1183,963,9,0,2,9,5,2289,2043,20,2,0,6,3,917,1006,14,0,0,6,3,1344,933,3,0,0,9,3,1122,949,5,1,0,10,4,1847,1866,25,1,0,10,3,841,1015,2,1,0,9,3,1091,918,6,0,1,8,5,1674,1048,12,0,1,7,1.0,0.0,72.0,-82.0,1000.0,2,2,3,0,12.0,247.0,-86.0,272.0,3,4,2,0,118.0,0,93,102,52,53,57,26,89,20,29,39,2,0
7,2,556,570,1,0,0,9,4,1194,1386,21,0,0,7,4,1374,1064,13,0,1,8,2,774,737,0,1,0,8,5,2254,1787,24,0,0,6,3,909,970,16,0,0,5,4,1543,929,10,0,0,7,4,1485,1389,19,0,0,14,3,1075,718,3,0,1,7,4,1130,994,1,1,0,10,84.0,1.0,138.0,-82.0,233.0,2,6,2,2,-29.0,168.0,-54.0,1000.0,3,2,2,1,16.0,1,20,6,65,92,43,54,34,93,99,5,9,0
10,2,751,808,1,0,0,13,2,421,569,3,0,0,6,3,960,896,3,1,2,6,5,1877,1511,23,0,0,8,4,1581,1593,26,0,0,4,3,687,675,1,0,0,1,2,515,793,4,0,0,8,3,964,1022,12,0,0,8,5,2434,1917,22,0,0,9,4,1372,2005,24,2,1,11,168.0,1.0,33.0,-88.0,1000.0,5,4,3,1,-22.0,46.0,-87.0,186.0,1,3,3,0,-34.0,1,112,26,29,11,41,91,22,10,49,54,9,0
13,3,708,903,1,1,1,11,2,672,901,6,0,0,8,5,2141,1626,28,0,0,7,4,1579,2169,28,2,0,7,5,1960,1668,27,0,0,8,4,1255,704,5,0,1,6,2,490,648,1,0,1,9,4,1308,1355,22,0,0,7,2,536,692,0,1,1,6,4,1815,1424,18,0,0,5,55.0,0.0,208.0,-78.0,180.0,4,2,2,2,-49.0,30.0,-89.0,210.0,3,4,2,1,-26.0,1,27,91,74,52,2,110,25,77,62,49,3,0
16,4,1259,661,4,0,0,9,5,1703,964,10,0,1,6,3,1099,942,2,1,0,6,3,1285,1013,10,0,0,8,2,484,609,4,0,0,8,4,2273,1680,16,1,0,6,2,511,541,1,0,0,4,4,1135,1399,19,0,0,10,3,623,520,1,0,0,10,5,1884,1409,15,0,1,11,285.0,1.0,225.0,-76.0,1000.0,1,3,2,0,36.0,180.0,-86.0,180.0,1,3,2,1,-33.0,1,39,93,30,85,102,14,64,70,3,96,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114369,5,2054,1941,27,0,1,8,4,1460,936,7,0,1,8,3,1208,1579,22,0,1,13,2,464,816,1,0,1,10,3,576,932,3,1,0,6,2,557,683,2,0,0,10,2,786,652,2,0,1,4,3,1479,2369,19,4,0,9,4,2042,1287,12,0,0,5,3,1143,742,5,0,0,8,27.0,1.0,110.0,-84.0,180.0,5,3,4,1,8.0,253.0,-87.0,1000.0,4,3,2,1,-33.0,7,11,28,89,20,86,84,97,8,74,37,7,1
114377,3,748,605,1,0,0,12,3,1130,1385,16,1,0,9,3,779,660,0,0,0,8,4,1463,1362,18,0,0,8,4,1549,1479,24,0,1,8,5,1931,1559,13,1,0,8,3,931,576,2,0,0,8,4,1678,1349,18,0,0,7,3,633,500,0,0,0,10,4,1197,1245,18,0,1,8,147.0,1.0,1000.0,-82.0,182.0,2,4,2,1,-1.0,133.0,-85.0,184.0,2,3,4,1,-18.0,1,3,22,60,11,8,28,112,46,31,72,5,4
114378,2,575,499,0,0,0,8,3,697,686,3,0,0,8,5,2050,980,8,0,1,8,3,853,1014,13,0,1,6,4,1325,1390,21,0,1,11,3,844,877,9,0,0,11,3,702,556,0,0,0,10,3,1073,1004,2,1,0,10,5,1622,1684,22,1,0,10,5,2104,1963,25,1,0,8,4.0,1.0,10

In [81]:
submission = pd.read_csv(r'S:\Coursera\Введение в машинное обучение\Итоговое задание\submission.csv')

In [82]:
submission['match_id'] = X_test_test.index

In [83]:
submission['radiant_win'] = best_model.predict_proba(X_test_test)[:,1]

In [84]:
submission.to_csv(r'S:\Coursera\Введение в машинное обучение\Итоговое задание\my_submission_catboost.csv', index=False)

In [ ]:
4. Логистическая регрессия

In [68]:
#Готовим данные для лог. регрессии заново и немного по-другому

#one-hot-encoding:
X = pd.get_dummies(X, columns=cat_cols, drop_first=True)
X

r1_level  r1_xp  r1_gold  r1_lh  r1_kills  r1_deaths  r1_items  \
match_id                                                                   
0                5   2098     1489     20         0          0         7   
1                4   1188     1033      9         0          1        12   
2                4   1319     1270     22         0          0        12   
3                4   1779     1056     14         0          0         5   
4                4   1431     1090      8         1          0         8   
...            ...    ...      ...    ...       ...        ...       ...   
114402           4   1706     1198     17         0          1         8   
114403           4   1793     1416     17         0          1         5   
114404           4   1399      540      1         0          0         5   
114405           3   1135      766      6         0          2         6   
114406           3   1053      799      7         0          0         7   

          r2_level  r2_xp  r2_gold  r2_lh  r2_kills  r2_deaths  r2_items  \
match_id                                                                   
0                3    842      991     10         0          0         4   
1                4   1596      993     10         0          1         7   
2                3   1314      775      6         0          0         6   
3                2    539      539      1         0          0         6   
4                2    629      552      0         0          1         7   
...            ...    ...      ...    ...       ...        ...       ...   
114402           2    616      535      0         0          2        10   
114403           3    764      800      0         1          0         9   
114404           4   1448     1371     17         0          1        13   
114405           5   1954     1457     22         0          0         9   
114406           5   2097     1818     23         1          2         9   

          r3_level  r3_xp  r3_gold  r3_lh  r3_kills  r3_deaths  r3_items  \
match_id                                                                   
0                5   1909     1143     10         0          0         8   
1                4   1506     1502     18         1          0         7   
2                3   1297      909      0         1          0         6   
3                5   2037     1139     15         0          0         6   
4                3    884      927      0         1          0         8   
...            ...    ...      ...    ...       ...        ...       ...   
114402           3    879     1798     22         1          1         9   
114403           3   1130     1585     20         1          1        14   
114404           3    658      646      2         0          0        10   
114405           2    471      634      2         0          1        10   
114406           3   1103      889      3         1          0         9   

          r4_level  r4_xp  r4_gold  r4_lh  r4_kills  r4_deaths  r4_items  \
match_id                                                                   
0                3    757      741      6         0          0         7   
1                3    669      631      7         0          0         7   
2                5   2360     2096     26         1          1         6   
3                2    591      499      0         0          0         6   
4                3    925     1439     16         1          0        11   
...            ...    ...      ...    ...       ...        ...       ...   
114402           4   1835     1350     11         1          1         9   
114403           4   1546     1166      8         1          0         6   
114404           4   1710     1833     24         1          0        10   
114405           1    265      605      1         0          0         9   
114406           4   1293     1675     11         1          0        11   

          r5_level  r5_xp  r5_gold  r5_lh  r5_kills  r

In [73]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV

In [70]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2)

In [71]:
#масштабируем признаки

pca = StandardScaler()
X_train_ = pca.fit_transform(X_train)
X_test_ = pca.fit_transform(X_test)

In [74]:
#дефолтная лог. регрессия

logreg = LogisticRegression(random_state=2, max_iter=500)
logreg.fit(X_train_, y_train)

LogisticRegression(max_iter=500, random_state=2)

In [75]:
Pred_logreg_test = logreg.predict_proba(X_test_)
Pred_logreg_train = logreg.predict_proba(X_train_)
roc_auc_test = roc_auc_score(y_test, Pred_logreg_test[:,1])
roc_auc_train = roc_auc_score(y_train, Pred_logreg_train[:,1])
print('значение roc_auc на test: ', roc_auc_test)
print('значение roc_auc на train: ', roc_auc_train)
print('среднее значение roc_auc: ', (roc_auc_test + roc_auc_train)/2)

значение roc_auc на test:  0.7445053882786752
значение roc_auc на train:  0.7632273314137803
среднее значение roc_auc:  0.7538663598462277


In [ ]:
Вывод: 1. Переобучение незначительно 2. Точность выше, чем у кэтбустинга

In [ ]:
5. Логистическая регрессия с поиском лучших параметров

In [76]:
a = np.linspace(0.001, 1000, 100)
logregcv = LogisticRegressionCV(Cs=a , fit_intercept=True, cv=6, scoring='roc_auc', random_state=2, refit=True, max_iter=500)

In [77]:
logregcv.fit(X_train_, y_train)

LogisticRegressionCV(Cs=array([1.00000e-03, 1.01020e+01, 2.02030e+01, 3.03040e+01, 4.04050e+01,
       5.05060e+01, 6.06070e+01, 7.07080e+01, 8.08090e+01, 9.09100e+01,
       1.01011e+02, 1.11112e+02, 1.21213e+02, 1.31314e+02, 1.41415e+02,
       1.51516e+02, 1.61617e+02, 1.71718e+02, 1.81819e+02, 1.91920e+02,
       2.02021e+02, 2.12122e+02, 2.22223e+02, 2.32324e+02, 2.42425e+02,
       2.52526e+02, 2.62627...
       7.57576e+02, 7.67677e+02, 7.77778e+02, 7.87879e+02, 7.97980e+02,
       8.08081e+02, 8.18182e+02, 8.28283e+02, 8.38384e+02, 8.48485e+02,
       8.58586e+02, 8.68687e+02, 8.78788e+02, 8.88889e+02, 8.98990e+02,
       9.09091e+02, 9.19192e+02, 9.29293e+02, 9.39394e+02, 9.49495e+02,
       9.59596e+02, 9.69697e+02, 9.79798e+02, 9.89899e+02, 1.00000e+03]),
                     cv=6, max_iter=500, random_state=2, scoring='roc_auc')

In [78]:
Pred_logregcv_test = logregcv.predict_proba(X_test_)
Pred_logregcv_train = logregcv.predict_proba(X_train_)
roc_auc_test = roc_auc_score(y_test, Pred_logregcv_test[:,1])
roc_auc_train = roc_auc_score(y_train, Pred_logregcv_train[:,1])
print('значение roc_auc на test: ', roc_auc_test)
print('значение roc_auc на train: ', roc_auc_train)
print('среднее значение roc_auc: ', (roc_auc_test + roc_auc_train)/2)

значение roc_auc на test:  0.7445071633997126
значение roc_auc на train:  0.7632264054002934
среднее значение roc_auc:  0.753866784400003


In [ ]:
Вывод: 1. Переобучение незначительно 2. Точность почти не изменилась 3. Результат на каггле 0.74749

ПРЕДСКАЗАНИЯ ДЛЯ ЛОГ. РЕГРЕССИЙ

In [94]:
best_model = logregcv

In [97]:
X_test_test = pd.read_csv(r'S:\Coursera\Введение в машинное обучение\Итоговое задание\features_test.csv', index_col='match_id')
X_test_test = X_test_test[feature_cols]
X_test_test['first_blood_player1'] = X_test_test['first_blood_player1'].fillna(0)
X_test_test['first_blood_player2'] = X_test_test['first_blood_player2'].fillna(0)
X_test_test['first_blood_team'] = X_test_test['first_blood_team'].fillna(.5)
X_test_test[time_gap_cols] = X_test_test[time_gap_cols].fillna(1000)
X_test_test = cat_to_int(X_test_test, cat_cols)
X_test_test = pd.get_dummies(X_test_test, columns=cat_cols, drop_first=True)
X_test_test_ = pca.fit_transform(X_test_test)
X_test_test_

array([[ 0.47474566, -0.23774298, -0.1370936 , ..., -0.22689511,
        -0.23528192, -0.23181681],
       [-1.36563036, -1.20791332, -1.25566036, ..., -0.22689511,
        -0.23528192, -0.23181681],
       [-1.36563036, -0.86205735, -0.74271453, ..., -0.22689511,
        -0.23528192, -0.23181681],
       ...,
       [-1.36563036, -1.17421453, -1.40868202, ..., -0.22689511,
        -0.23528192, -0.23181681],
       [ 0.47474566,  1.0765097 ,  0.05041181, ..., -0.22689511,
        -0.23528192, -0.23181681],
       [ 0.47474566, -0.0390975 ,  0.32843708, ..., -0.22689511,
        -0.23528192, -0.23181681]])

In [98]:
submission = pd.read_csv(r'S:\Coursera\Введение в машинное обучение\Итоговое задание\submission.csv')
submission['match_id'] = X_test_test.index
submission['radiant_win'] = best_model.predict_proba(X_test_test_)[:,1]

In [99]:
submission.to_csv(r'S:\Coursera\Введение в машинное обучение\Итоговое задание\my_submission_logreg.csv', index=False)